## Relocation

In [66]:
"""SMART.ipynb

Three level sequential logit model.
Latent variables.
Maximum likelihood (full information) estimation.

:author: Tukey
:date: Wed Jan 20 08:22:28 2021

"""
import pandas as pd
import math
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.distributions as dist
import biogeme.results as res
import biogeme.optimization as opt
import biogeme.messaging as msg
from biogeme.expressions import bioDraws, Beta, DefineVariable, RandomVariable, log, RandomVariable, Integrate, Elem, bioNormalCdf, log, PanelLikelihoodTrajectory, MonteCarlo, exp

In [67]:
# Read the data
RELO = pd.read_csv("RELO.csv",sep=',',index_col = 0)

In [68]:
print(RELO.shape)
RELO = RELO.dropna()
print(RELO.shape)

(568, 74)
(568, 74)


In [69]:
pd.set_option('display.max_columns', None)
RELO.head()

,ID,AGE,COMP_Uber,COMP_Lyft,WEEK.TRIPS,WEEK.TRIPS.RJ,GENDER,HISPANIC,RACE,IMMG,EDU,STDT,EMPL,ZIPCODE,HHSIZE,CHILD,HHINCOME,INDVINCOME,WKHOURS,TGINCOME,SOURCE,POVERTY,WT,RL,ADI,ADH,DMLS,DMLW,DMDS_1,DMDS_2,DMDS_3,DMDS_4,DMDW_1,DMDW_2,DMDW_3,DMDW_4,RTD_1,RTD_2,RTD_3,RTD_4,obs,DTGINCOME,DTGWKHOURS,ADI_TGINCOME,ADH_WKHOURS,INCOME_TG,WKHOURS_TG,ER,SURGERATE1,SURGERATE2,SURGERATE3,SURGERATE4,WTRATE1,WTRATE2,WTRATE3,WTRATE4,surgeRank3,surgeRank1,surgeRank2,surgeRank4,surgeRank5,RTDRank1,RTDRank2,RTDRank3,RTDRank4,WTRank3,WTRank1,WTRank2,WTRank4,WTRank5,RTD10_1,RTD10_2,RTD10_3,RTD10_4
2,1,1,0,1,15,0,0,1,6,0,2,0,2,98126,2,0,9,8,10,250,3,1,1,2,400.0,4.0,0,16,8,8,0,16,0,0,30,0,30,16,30,30,5,1,1,1.6,0.4,1,0,100.000,0.266667,0.5,0.000000,0.533333,0.000000,0.000000,1.0,0.0,4,2,2,4,1,2,1,2,2,4,1,1,5,1,1,1,1,1
1,1,1,0,1,15,0,0,1,6,0,2,0,2,98126,2,0,9,8,10,250,3,1,1,3,175.0,7.0,16,0,12,0,8,0,0,30,0,8,4,8,30,4,4,1,1,0.7,0.7,0,0,25.000,3.000000,0.0,0.266667,0.000000,0.000000,3.750000,0.0,2.0,1,2,4,3,4,1,3,4,1,1,1,5,1,4,0,0,1,0
3,3,0,1,1,45,6,0,0,2,1,2,0,2,98043,1,0,7,7,10,300,3,0,1,3,210.0,4.0,12,0,0,0,0,16,30,24,0,0,16,30,8,4,1,1,1,0.7,0.4,0,0,52.500,0.000000,0.0,0.000000,4.000000,1.875000,0.800000,0.0,0.0,2,3,3,3,1,3,4,2,1,1,5,4,1,1,1,1,0,0
4,3,0,1,1,45,6,0,0,2,1,2,0,2,98043,1,0,7,7,10,300,3,0,1,3,210.0,16.0,0,16,0,0,8,16,16,16,0,0,12,30,12,12,5,1,1,0.7,1.6,0,1,13.125,0.000000,0.0,0.666667,1.333333,1.333333,0.533333,0.0,0.0,3,3,3,2,1,1,4,1,1,3,3,3,1,1,1,1,1,1
5,4,0,0,1,40,20,0,0,2,1,1,0,2,98188,1,0,7,7,8,0,3,0,1,2,70.0,5.6,0,30,16,8,16,0,0,0,0,0,16,4,30,4,1,0,1,0.0,0.7,1,0,12.500,1.000000,2.0,0.533333,0.000000,0.000000,0.000000,0.0,0.0,4,1,3,1,4,3,1,4,1,5,1,1,1,1,1,0,1,0


In [70]:
RELO_database = db.Database("RELO.csv",RELO)

#from headers import *
# The following statement allows you to use the names of the variable
# as Python variable.
globals().update(RELO_database.variables)

In [71]:
## describe the data
pd.set_option('display.max_columns', None)
RELO.describe()

,ID,AGE,COMP_Uber,COMP_Lyft,WEEK.TRIPS,WEEK.TRIPS.RJ,GENDER,HISPANIC,RACE,IMMG,EDU,STDT,EMPL,ZIPCODE,HHSIZE,CHILD,HHINCOME,INDVINCOME,WKHOURS,TGINCOME,SOURCE,POVERTY,WT,RL,ADI,ADH,DMLS,DMLW,DMDS_1,DMDS_2,DMDS_3,DMDS_4,DMDW_1,DMDW_2,DMDW_3,DMDW_4,RTD_1,RTD_2,RTD_3,RTD_4,obs,DTGINCOME,DTGWKHOURS,ADI_TGINCOME,ADH_WKHOURS,INCOME_TG,WKHOURS_TG,ER,SURGERATE1,SURGERATE2,SURGERATE3,SURGERATE4,WTRATE1,WTRATE2,WTRATE3,WTRATE4,surgeRank3,surgeRank1,surgeRank2,surgeRank4,surgeRank5,RTDRank1,RTDRank2,RTDRank3,RTDRank4,WTRank3,WTRank1,WTRank2,WTRank4,WTRank5,RTD10_1,RTD10_2,RTD10_3,RTD10_4
count,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.0,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000,568.000000
mean,91.466549,0.519366,0.598592,0.852113,52.830986,3.663732,0.065141,0.075704,2.376761,0.975352,1.522887,0.114437,1.670775,97362.096831,3.385563,1.232394,6.248239,5.843310,6.283451,151.619718,3.0,0.102113,0.982394,2.963028,181.852113,6.749824,4.309859,9.035211,4.781690,4.838028,4.338028,4.746479,8.035211,8.933099,9.028169,8.334507,14.450704,14.862676,13.926056,14.320423,3.492958,0.698944,0.709507,0.696303,0.550528,0.730634,0.554577,33.540824,0.527171,0.491549,0.514906,0.585211,0.779754,0.905428,0.941447,0.916828,2.362676,2.334507,2.336268,2.387324,2.380282,2.301056,2.269366,2.202465,2.154930,2.359155,2.198944,2.322183,2.286972,2.357394,0.656690,0.630282,0.566901,0.556338
std,53.211292,0.566226,0.490615,0.355301,31.537603,5.302286,0.267557,0.350723,1.190329,0.236281,0.679379,0.383895,0.649834,8270.324230,2.126743,1.736284,1.906083,1.723913,4.367461,112.471120,0.0,0.303063,0.131629,1.189477,109.059059,3.296072,5.762107,11.271798,5.457052,5.780387,5.616664,6.168060,11.191400,11.308521,11.582961,11.073304,9.049016,9.197547,9.200193,9.811062,1.687161,0.459121,0.454390,0.574168,0.459007,0.444022,0.497450,26.958795,0.814978,0.700126,0.841814,1.034007,1.390323,1.645435,1.642822,1.615742,1.034918,1.071059,1.090906,1.041289,1.154593,1.099558,1.143795,1.098112,1.139608,1.606168,1.637587,1.566875,1.578193,1.655757,0.475232,0.483154,0.495941,0.497254
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,9188.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,3.0,0.000000,0.000000,1.000000,40.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,4.000000,4.000000,4.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.564103,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,45.000000,0.000000,0.000000,1.000000,30.000000,0.000000,0.000000,0.000000,2.000000,1.000000,1.000000,0.000000,2.000000,98092.000000,1.000000,0.000000,5.000000,5.000000,0.000000,0.000000,3.0,0.000000,1.000000,2.000000,100.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,8.000000,8.000000,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.750000,1.000000,1.000000,2.000000,1.000000,1.000000,1.0000

### Lower level: relocation mixed logit model

In [94]:
# Parameters to be estimated
# one cost/waiting time
ASC_1 = Beta('ASC_1', 0.05, -1000, 1000, 0) ## No, stay at the same place
ASC_2 = Beta('ASC_2', 0.05, -1000, 1000, 0) ## No, stay at the same place
ASC_3 = Beta('ASC_3', 0.05, -1000, 1000, 0) ## No, stay at the same place
ASC_4 = Beta('ASC_4', 0.05, -1000, 1000, 0) ## No, stay at the same place


B_ADI = Beta('B_ADI', 0.05, -1000, 1000, 0) 
B_ADH = Beta('B_ADH', 0.05, -1000, 1000, 0) 

B_GENDER = Beta('B_GENDER', 0.05, -1000, 1000, 0) 
B_AGE = Beta('B_AGE', 0.05, -1000, 1000, 0) 
B_EMPL = Beta('B_EMPL', 0.05, -1000, 1000, 0) 

B_DMDS = Beta('B_DMLS', 0.05, -1000, 1000, 0) 
B_DMDW = Beta('B_DMLW', 0.05, -1000, 1000, 0) 

B_RTD = Beta('B_RTD_1', 0.05, -1000, 1000, 0) 

## definition of the utility functions
No_1 = B_DMDS*DMDS_1 + B_RTD*RTD_1
No_2 = B_DMDS*DMDS_2 + B_RTD*RTD_2
No_3 = ASC_3 + B_ADI*ADI + B_DMDS*DMLS + B_GENDER*GENDER
No_4 = B_DMDS*DMDS_3 + B_RTD*RTD_3
No_5 = B_DMDS*DMDS_4 + B_RTD*RTD_4

In [95]:
## Associate the utility functions with the numbering of the alternatives
V_mode_1 = {
    1:No_1,
    2:No_2,
    3:No_3,
    4:No_4,
    5:No_5
}

## Availability of each alternative, as described in the dataset
Mode_1 = {
    1:1,
    2:1, 
    3:1,  
    4:1,
    5:1
}

# Definition of the model
logprob_RELO = models.logit(V_mode_1, Mode_1, RL)

# Create the Biogeme object
biogeme_RELO = bio.BIOGEME(RELO_database, logprob_RELO)
biogeme_RELO.modelName = 'Relocation choice'

# Estimate the parameters
results_RELO = biogeme_RELO.estimate(algoParameters={'tolerance':0.0001,'maxiter':10000})

# Get the results in a pandas table
RELO_result = results_RELO.getEstimatedParameters()
RELO_result

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_3,14.709729,5.042172e+01,2.917340e-01,0.770490,3.366567e+00,4.369356e+00,0.000012
B_ADH,34.759201,4.307116e+01,8.070180e-01,0.419656,3.916420e+00,8.875248e+00,0.000000
B_ADI,-1.255543,1.643459e+00,-7.639637e-01,0.444889,1.495227e-01,-8.397011e+00,0.000000
B_DMLS,92.234185,1.174783e+02,7.851170e-01,0.432385,1.082320e+01,8.521898e+00,0.000000
B_GENDER,5.878736,9.749113e-10,6.030021e+09,0.000000,8.960974e-11,6.560376e+10,0.000000
B_RTD_1,-50.929347,6.572608e+01,-7.748727e-01,0.438415,6.049179e+00,-8.419216e+00,0.000000


In [15]:
# The choice model is a logit, with availability conditions
prob_Walking1 = models.logit(V_mode_1, av_mode_1, 1)
prob_Biking1 = models.logit(V_mode_1, av_mode_1, 2)
prob_Transit1 = models.logit(V_mode_1, av_mode_1, 3)
prob_Ridehailing1 = models.logit(V_mode_1, av_mode_1, 4)
prob_SD_ridehailing1 = models.logit(V_mode_1, av_mode_1, 5)
prob_Car1 = models.logit(V_mode_1, av_mode_1, 6)
prob_SD_car1 = models.logit(V_mode_1, av_mode_1, 7)

simulate = {'Walking_1': prob_Walking1,
            'Biking_1': prob_Biking1,
            'Transit_1': prob_Transit1,
            'Ridehailing_1': prob_Ridehailing1,
            'SD_ridehailing_1': prob_SD_ridehailing1,
            'Car_1': prob_Car1,
            'SD_car_1': prob_SD_car1}
                      
biogeme = bio.BIOGEME(database, simulate)
SV_trip1 = biogeme.simulate(results_trip1.getBetaValues())

##print(simulatedValues)
pd.set_option('display.max_rows', None)
SV_trip1.mean()

NameError: name 'database' is not defined

### Upper level

In [22]:
## calculate mode choice utilities
Car = MDresult.loc["B_CAR_TT_1", "Value"]*T1CTT + MDresult.loc["B_PARKING1", "Value"]*T1CPF
SD_car = MDresult.loc["ASC_SDC_1", "Value"] + MDresult.loc["B_SDC_TT_1", "Value"]*T1ATT + MDresult.loc["B_PARKING1", "Value"]*T1APF
Walking = MDresult.loc["ASC_WK_1", "Value"] + MDresult.loc["B_WK_TT_1", "Value"]*T1WTT
Biking = MDresult.loc["ASC_BK_1", "Value"] + MDresult.loc["B_BK_TT_1", "Value"]*T1BTT
Transit = MDresult.loc["ASC_TS_1", "Value"] + MDresult.loc["B_TS_TT_1", "Value"]*T1TTT + MDresult.loc["B_TS_WT_1", "Value"]*T1TWT + MDresult.loc["B_COST_1", "Value"]*T1TTC
Ridehailing = MDresult.loc["ASC_RH_1", "Value"] + MDresult.loc["B_RH_TT_1", "Value"]*T1RTT + MDresult.loc["B_RH_WT_1", "Value"]*T1RWT + MDresult.loc["B_COST_1", "Value"]*T1RTC
SD_ridehailing = MDresult.loc["ASC_SDRH_1", "Value"] + MDresult.loc["B_SDRH_TT_1", "Value"]*T1ARTT + MDresult.loc["B_SDRH_WT_1", "Value"]*T1RWT + MDresult.loc["B_COST_1", "Value"]*T1ARTC

## 0 cars, 0 SD cars
## Define scale parameters
lambda1 = Beta('lambda1', 0.1, 0.1, 1, 0) ## scale parameter for nest 1
lambda2 = Beta('lambda2', 0.1, 0.1, 1, 0) ## scale parameter for nest 2
lambda3 = Beta('lambda3', 0.1, 0.1, 1, 0) ## scale parameter for nest 3
lambda4 = Beta('lambda4', 0.1, 0.1, 1, 0) ## scale parameter for nest 4

## Define car ownership level intercepts
ASC_zero_one = Beta('ASC_zero_one', 0, -10, 10, 0) ## biking
ASC_one_zero = Beta('ASC_one_zero', 0, -10, 10, 0) ## transit
ASC_one_one = Beta('ASC_one_one', 0, -10, 10, 0) ## ridehailing

## Define coefficient for monthly payment
B_MP = Beta('B_MP', 0.5, -1000, 1000, 0) 

## Define utility function
## zero cars, zero driverless cars
PV00 = 0
RH = log(\
           exp(Ridehailing*MDresult.loc["MU_RH", "Value"])*av_rh_trip1 + \
           exp(SD_ridehailing*MDresult.loc["MU_RH", "Value"])*av_arh_trip1\
          )
OT = log(\
           exp(Walking) + \
           exp(Biking) + \
           exp(Transit)*av_transit_trip1
          )
IV00 = log(\
        exp(RH/lambda1)+\
        exp(OT/lambda1)
        )

## zero cars, one driverless car
PV01 = SD_car*MDresult.loc["MU_car", "Value"]
IV01 = log(\
        exp(PV01/lambda2) + \
        exp(RH/lambda2)+\
        exp(OT/lambda2)
        )

## one car, zero driverless car
PV10 = Car*MDresult.loc["MU_car", "Value"]

IV10 = log(\
        exp(PV10/lambda3) + \
        exp(RH/lambda3)+\
        exp(OT/lambda3)
        )

## one car, one driverless car
PV11 = log(\
           exp(Car*MDresult.loc["MU_car", "Value"]) + \
           exp(SD_car*MDresult.loc["MU_car", "Value"])\
          )

IV11 = log(\
        exp(PV11/lambda4) + \
        exp(RH/lambda4)+\
        exp(OT/lambda4)
        )

## Total utility of each car ownership level
zero_zero = IV00 + B_MP*MP_zero_zero
zero_one =  ASC_zero_one + IV01 + B_MP*MP_zero_one
one_zero =  ASC_one_zero + IV10 + B_MP*MP_one_zero
one_one =  ASC_one_one + IV11 + B_MP*MP_one_one

## Associate the utility functions with the numbering of the alternatives
V_profile = {
        1:zero_zero,
        2:zero_one,
        3:one_zero,
        4:one_one
        }

av_profile = {
        1:av_zero_zero,
        2:av_zero_one,
        3:av_one_zero,
        4:av_one_one
        }

# Definition of the model
logprob_trip = models.loglogit(V_profile, av_profile, PChoice)

# Create the Biogeme object
biogeme_logprob_trip = bio.BIOGEME(database, logprob_trip)
biogeme_logprob_trip.modelName = 'car ownership-nl-mnl'

# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage(2)
logger.setDetailed()

# Estimate the parameters
results_trip = biogeme_logprob_trip.estimate()

# Get the results in a pandas table
pandasResults_trip = results_trip.getEstimatedParameters()

pandasResults_trip

[12:32:19] < Detailed >  Log likelihood (N = 1814):  -51690.28
[12:32:19] < Detailed >  ** Optimization: BFGS with trust region for simple bounds
[12:32:19] < General >   Log likelihood (N = 1814):  -51690.28 Gradient norm:        inf  
[12:32:19] < Detailed >  Log likelihood (N = 1814):  -45854.46
[12:32:19] < Detailed >  1 f=  28.49519 projected rel. grad.=    35 delta=   0.5 rho=8.1e-306 -
[12:32:19] < Detailed >  Log likelihood (N = 1814):  -47227.12
[12:32:19] < Detailed >  2 f=  28.49519 projected rel. grad.=    35 delta=  0.25 rho=1.2e-305 -
[12:32:19] < Detailed >  Log likelihood (N = 1814):   -49356.1
[12:32:19] < Detailed >  3 f=  28.49519 projected rel. grad.=    35 delta=  0.12 rho=1.3e-305 -
[12:32:19] < Detailed >  Log likelihood (N = 1814):  -50509.07
[12:32:19] < Detailed >  4 f=  28.49519 projected rel. grad.=    35 delta= 0.062 rho=1.3e-305 -
[12:32:19] < Detailed >  Log likelihood (N = 1814):  -51096.86
[12:32:19] < Detailed >  5 f=  28.49519 projected rel. grad.=   

/Users/tuyuanjie/opt/anaconda3/lib/python3.7/site-packages/biogeme/optimization.py:448: RuntimeWarning: invalid value encountered in double_scalars
  ratio = -fprime / fsecond
/Users/tuyuanjie/opt/anaconda3/lib/python3.7/site-packages/biogeme/optimization.py:464: RuntimeWarning: overflow encountered in double_scalars
  dg = np.sum([d[i] * g[i] for i in ind])
/Users/tuyuanjie/opt/anaconda3/lib/python3.7/site-packages/biogeme/optimization.py:465: RuntimeWarning: invalid value encountered in double_scalars
  fprime += delta_t * fsecond - np.inner(b, x) - dg
/Users/tuyuanjie/opt/anaconda3/lib/python3.7/site-packages/biogeme/optimization.py:466: RuntimeWarning: invalid value encountered in double_scalars
  fsecond += np.inner(b, bd - 2 * d)


[12:32:20] < Detailed >  Log likelihood (N = 1814):  -51687.95
[12:32:20] < Detailed >  13 f=  28.49519 projected rel. grad.=    35 delta=0.00012 rho=1.3e-305 -
[12:32:20] < Detailed >  Log likelihood (N = 1814):  -51689.12
[12:32:20] < Detailed >  14 f=  28.49519 projected rel. grad.=    35 delta=6.1e-05 rho=1.3e-305 -
[12:32:20] < Detailed >  Log likelihood (N = 1814):   -51689.7
[12:32:20] < Detailed >  15 f=  28.49519 projected rel. grad.=    35 delta=3.1e-05 rho=1.3e-305 -
[12:32:20] < Detailed >  Log likelihood (N = 1814):  -51689.99
[12:32:20] < Detailed >  16 f=  28.49519 projected rel. grad.=    35 delta=1.5e-05 rho=1.3e-305 -
[12:32:20] < Detailed >  Log likelihood (N = 1814):  -51690.13
[12:32:20] < Detailed >  17 f=  28.49519 projected rel. grad.=    35 delta=7.6e-06 rho=1.3e-305 -
[12:32:20] < Detailed >  Log likelihood (N = 1814):  -51690.21
[12:32:20] < Detailed >  18 f=  28.49519 projected rel. grad.=    35 delta=3.8e-06 rho=1.3e-305 -
[12:32:20] < Detailed >  Log likel

,Value,Active bound,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_one_one,0.0,0.0,9.322926e-156,0.000000e+00,1.0,1.797693e+308,0.000000e+00,1.0
ASC_one_zero,0.0,0.0,9.322926e-156,0.000000e+00,1.0,1.797693e+308,0.000000e+00,1.0
ASC_zero_one,0.0,0.0,9.322926e-156,0.000000e+00,1.0,1.797693e+308,0.000000e+00,1.0
B_MP,0.5,0.0,9.322926e-156,5.363123e+154,0.0,1.797693e+308,2.781342e-309,1.0
lambda1,0.1,1.0,9.322926e-156,1.072625e+154,0.0,1.797693e+308,5.562685e-310,1.0
lambda2,0.1,1.0,9.322926e-156,1.072625e+154,0.0,1.797693e+308,5.562685e-310,1.0
lambda3,0.1,1.0,9.322926e-156,1.072625e+154,0.0,1.797693e+308,5.562685e-310,1.0
lambda4,0.1,1.0,9.322926e-156,1.072625e+154,0.0,1.797693e+308,5.562685e-310,1.0
